In [12]:
import pandas as pd
import torch
import torch.nn as nn
finalstatmatrix = pd.read_csv("data/modelmatrix.csv")
finalstatmatrix.set_index('Unnamed: 0',inplace=True)
finalstatmatrix = finalstatmatrix.transpose().fillna(0)
double = [30,37,108,256,327,384,390,459,571,643,645,687,716,724,756,927,931,1060,1062,1140,1189,1281,1283,1354,1355,1383,1395,1396,1436,1438,1442,1443,1463,1559,
 1597,1616,1642,1646,1658,1664,1684,1685,1747,1806,1849,1890,2039,2188,2264,2320,2342,2599,2622,2649,2691,2757,2763,2801,2840,2845,2846,2892,2913,2971,
 3043,3143,3150,3169,3181,3233,3266,3267,3291,3379,3412,3418,3423,3559,3601,3620,3652,3716,3718,3722,3770,3784,3840,3852,3861,3945,4040,4065,4159,4184,
 4237,4279,4286,4296,4305,4371,4373,4392,4398,4480,4481,4518,4546,4634,4845,4923,4952,4961,5006,5038,5085,5089,5091,5173,5185,5251,5252,5266,5309,5329,
 5332,5470,5583,5656,5662,5747,5911,5953,5987,6010,6136,6161,6191,6301,6320,6330,6383,6418,6460,6519,6523,6539,6575,6717,6777,6783,6792,6815,6816,6818,
 6880,6895,6949,6980,7020,7093,7133,7158,7307,7442,7486,7579,7589,7599,7649,7678,7787,7799,7852,7866,7927,8038,8206,8257,8258,8274,8287,8464,8495,8586,
 8616,8635,8638,8641,8642,8643,8710,8735,8743,8751,8778,8816,8818,8868,8955,8956,8962,8981,8988,9023]
match = pd.read_csv("data/match.csv")
match = match.drop(double)
X = torch.Tensor(finalstatmatrix.values)
y1 = torch.Tensor(match["Score"].str.split("–",expand=True).iloc[:,0].astype(int))[:,None]
y2 = torch.Tensor(match["Score"].str.split("–",expand=True).iloc[:,1].astype(int))[:,None]


In [ ]:
# Simple Linear Regression and Logisitc Regression
X = X.nan_to_num(nan=0)
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVC, LinearSVC
mod1 = LinearRegression().fit(X,y1)
mod2 = LinearRegression().fit(X,y2)
mod3 = LogisticRegression().fit(X,y1)
mod4 = LogisticRegression().fit(X,y2)
mod5 = SVC().fit(X,y1)
mod6 = SVC().fit(X,y2)
mod7 = LinearSVC(C=0.025).fit(X,y1)
mod8 = LinearSVC(C=0.025).fit(X,y2)
print("Linear Regression",(mod1.score(X,y1)+mod2.score(X,y2))/2)
print("Logisitic Regression:", (mod3.score(X,y1)+mod4.score(X,y2))/2)
print("SVM with RBF Kernel:", (mod5.score(X,y1)+mod6.score(X,y2))/2)
print("Linear SVM: ", (mod7.score(X,y1)+mod8.score(X,y2))/2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
mod9 = DecisionTreeClassifier(max_depth=5).fit(X,y1)
mod10 = DecisionTreeClassifier(max_depth=5).fit(X,y1)
mod11 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1).fit(X,y1)
mod12 = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1).fit(X,y2)
mod13 = MLPClassifier(alpha=1, max_iter=1000).fit(X,y1)
mod14 = MLPClassifier(alpha=1, max_iter=1000).fit(X,y2)
print("Decision Tree ", (mod9.score(X,y1)+mod10.score(X,y2))/2)
print("Random Forest: ", (mod11.score(X,y1)+mod12.score(X,y2))/2)
print("MLP Classifier:", (mod13.score(X,y1)+mod14.score(X,y2))/2)
print("KNN: ",(cross_val_score(KNeighborsClassifier(10), X, y1)+cross_val_score(KNeighborsClassifier(10), X, y2)/2).mean())

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
X = X.nan_to_num(nan=0)
print("XGBoost:", (cross_val_score(XGBClassifier(), X, y1)+cross_val_score(XGBClassifier(), X, y2))/2)

In [83]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
#get a toy dataset and preprocess
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X,y2 = finalstatmatrix,match["Score"].str.split("–",expand=True).iloc[:,0].astype(int)
X_train, X_test, y_train,y_test= train_test_split(X,y,train_size=0.8,random_state=3)

X_train = torch.Tensor(X_train.values)
X_test = torch.Tensor(X_test.values)
y_train = torch.Tensor(y_train.values).long()
y_test = torch.Tensor(y_test.values).long()

In [140]:
from torch.utils.data import DataLoader
bs = 1
train_loader = DataLoader(dataset = X_train, batch_size=bs)
train_labelloader = DataLoader(dataset = y_train, batch_size=bs)

In [132]:
import torch.nn.functional as F
class MLP1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2614, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = MLP1()
learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()
for i in range(30):
     
    for xi, yi in zip(train_loader,train_labelloader):
        optimizer.zero_grad()
        Z = model(xi)
        loss = loss_function(Z, yi)  
        loss.backward() 
        optimizer.step()
    print("Loss at iteration {}: {}".format(i, loss))




Loss at iteration 0: nan
Loss at iteration 1: nan


KeyboardInterrupt: 

In [143]:
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2614, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
model = MLP()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
loss_function = nn.CrossEntropyLoss()

for i in range(2):
    running_loss=0
    for xi, yi in zip(train_loader, train_labelloader):
        xi.na

12.152510643005371 tensor(12.1525, grad_fn=<NllLossBackward0>)
12.152510643005371 tensor(0., grad_fn=<NllLossBackward0>)
10446.368330955505 tensor(10434.2158, grad_fn=<NllLossBackward0>)
605928.8683309555 tensor(595482.5000, grad_fn=<NllLossBackward0>)
90558848.86833096 tensor(89952920., grad_fn=<NllLossBackward0>)
10978008093056.87 tensor(1.0978e+13, grad_fn=<NllLossBackward0>)
3.749138377879181e+16 tensor(3.7480e+16, grad_fn=<NllLossBackward0>)
5.756633668436767e+19 tensor(5.7529e+19, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward0>)
nan tensor(nan, grad_fn=<NllLossBackward

KeyboardInterrupt: 

In [110]:

loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.